# Imports

In [2]:
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision.datasets.folder import default_loader
import torchvision.transforms as T
import os
from torch.utils.data import random_split
from PIL import Image
import pandas as pd
import numpy as np
import glob
from torch import optim
from tqdm import tqdm, notebook # This is optional but useful
from torchvision import datasets
import matplotlib.pyplot as plt
import torch
from torch import nn

# Dataset

In [3]:
class Traffic(Dataset):
  def __init__(self, root_dir, transform=None, loader=default_loader):
    
    self.image_paths, self.labels = self._get_imgs_and_labels(root_dir)
    
    self.transform = transform
    self.loader = loader
    
  def __len__(self):
    return len(self.labels)
  
  def __getitem__(self, index):
    image = self.loader(self.image_paths[index])
    label = self.labels[index]

    if self.transform:
      image = self.transform(image)
  
    return image, torch.tensor(int(label)-1) # Convert labels to tensors, and subtract 1 for convention purposes (to start at 0)
  
  # Helper Functions
  def print_image(self, index):
    img = Image.open(self.image_paths[index])
    img.show()
    print(self.labels[index])

  def _get_imgs_and_labels(self, root_dir):
    image_full_path = []
    labels = []
    
    # Get paths to each image (in order)
    for img_path in sorted(glob.glob(os.path.join(root_dir, "images", "*.jpg"))):
      image_full_path.append(img_path)
    
    # Get labels
    with open(os.path.join(root_dir, "annotations.txt"), 'r') as f:
      for label in f:
        labels.append(int(label.strip()) + 1)
    
    return image_full_path, labels

In [ ]:
data_dir = "Traffic_Processed"

data = Traffic(root_dir=data_dir, transform=T.ToTensor())

data.print_image(10)

# Network

In [ ]:
class Convolutional(nn.Module):
  def __init__(self, in_channels=3):
    super().__init__()
    self.conv = nn.Sequential(nn.Conv2d(3, 32, kernel_size=3, padding=1, stride=1),
                              nn.ReLU(),
                              nn.MaxPool2d(kernel_size=2, stride=2),
                              
                              nn.Conv2d(32, 64, kernel_size=3, padding=1, stride=1),
                              nn.ReLU(),
                              nn.MaxPool2d(kernel_size=2, stride=2),
                              
                              nn.Conv2d(64, 128, kernel_size=3, padding=1, stride=1),
                              nn.ReLU(),
                              nn.MaxPool2d(kernel_size=2, stride=2),
                              
                              nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1, stride=1),
                              nn.ReLU(),
                              nn.MaxPool2d(kernel_size=2, stride=2),
                              
                              nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1, stride=1),
                              nn.ReLU(),
                              nn.MaxPool2d(kernel_size=2, stride=2),
                              
                              nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, padding=1, stride=1),
                              nn.ReLU(),
                              nn.MaxPool2d(kernel_size=2, stride=2),                                                
    )
    self.linear = nn.Sequential(nn.Dropout(p=0.5),
                                nn.Linear(in_features=4608, out_features=4608),
                                nn.ReLU(),
                                nn.Dropout(p=0.5),
                                nn.Linear(in_features=4608, out_features=7),
                                nn.Softmax(dim=1)
                                )
  
  def forward(self, x):
    x = self.conv(x)
    b, c, h, w = x.shape
    x = x.view(b, -1) # Flatten image
    x = self.linear(x)
    
    return x

# Data Processing

In [ ]:
data_dir = "./Traffic_Processed"

transform = T.Compose ([
  T.Resize((224, 224)),
  T.ToTensor(),
  T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

vehicle_data = Traffic(root_dir=data_dir, transform=transform)

train_data_size = int(len(vehicle_data) * .1) 
validation_data_size = int(len(vehicle_data) * .1)
other = len(vehicle_data) - (train_data_size + validation_data_size)

train_data, val_data, other = random_split(vehicle_data, [train_data_size, validation_data_size, other])

'''train = DataLoader(vehicle_data, batch_size=32, shuffle=True)

data = iter(train) # Let's iterate on it
single_point = next(data)
print(f"""Type: {type(single_point)}
Length: {len(single_point)}
More Types: {type(single_point[0])}, {type(single_point[1])}
Shapes: {single_point[0].shape}, {single_point[1].shape}
Labels: {single_point[1]}
""")

print(single_point[1][0])
print(single_point[1][1])
print(single_point[1][2])

import matplotlib.pyplot as plt
from torchvision import transforms as T
ToPIL = T.ToPILImage() # Converting function
img0 = ToPIL(single_point[0][0])
img1 = ToPIL(single_point[0][1])
# Plotting
fig, axs = plt.subplots(1,2)
axs[0].imshow(img0)
axs[1].imshow(img1)'''

# Training